In [64]:
import numpy as np
import random
import sys

data = np.loadtxt('assign1_data.txt')
#print data

x1 = data[:, 0]
print "x1:", x1

x2 = data[:, 1]
print "x2:", x2

y = data[:, 2]
#print "y:", y

z = data[:, 3]
#print "z:", z

x1: [ 0.1227  0.3914  0.7725  0.8342  0.5084  0.9983  0.6643  0.0954  0.6949
  0.6075  0.3717  0.0193  0.9314  0.2206  0.0903  0.5466  0.4331  0.5293
  0.8955  0.0701  0.0771  0.1972  0.6405  0.3203  0.103   0.5605  0.4288
  0.7101  0.6575  0.9498  0.555   0.0947  0.4293  0.31    0.7297  0.1973
  0.1571  0.2272  0.2099  0.3972  0.0033  0.6441  0.5549  0.9193  0.5341
  0.1555  0.1839  0.9949  0.7918  0.4088  0.2247  0.3901  0.4309  0.0732
  0.8077  0.0854  0.8955  0.8794  0.1094  0.0726  0.4377  0.889   0.5791
  0.9721  0.5719  0.3156  0.8065  0.4777  0.6264  0.2117  0.042   0.0916
  0.6035  0.6354  0.4102  0.695   0.2153  0.0768  0.2356  0.1591  0.7171
  0.9988  0.3936  0.0201  0.6295  0.1771  0.5888  0.5302  0.3434  0.7154
  0.3506  0.5432  0.4687  0.013   0.1311  0.697   0.3996  0.2208  0.1119
  0.4317]
x2: [ 0.299   0.6392  0.0826  0.0823  0.8025  0.7404  0.3861  0.986   0.605
  0.3618  0.761   0.7192  0.5181  0.1091  0.1085  0.2028  0.7049  0.688
  0.8477  0.6176  0.7883  0.6273  0

In [67]:
# create method to train and get result with different settings like online, minibatch, batch and learning rate
def train_and_get_result(x1, x2, lr):
    
    w1 = random.random() * 3
    w2 = random.random() * 3
    b = random.random() * 3

    print "random initial w1:\t", w1
    print "random initial w2:\t", w2
    print "random initial b:\t", b
    
    epoch = 0
    # training set size : validation set size = 3 : 1
    training_set_index = 0
    size = len(x1)
    validation_set_index = 0
    validation_set_index_step = size / 4
    pre_validation_error = sys.maxint
    curr_validation_error = 0
    while pre_validation_error >= curr_validation_error or curr_validation_error <= 0: # early stopping when the validation error is greater than its previous one
    #while True:

        validation_set_index += validation_set_index_step
        validation_set_index %= size

        x1_validation = x1[validation_set_index : validation_set_index + validation_set_index_step]
        x2_validation = x2[validation_set_index : validation_set_index + validation_set_index_step]
        y_validation = y[validation_set_index : validation_set_index + validation_set_index_step]

        # training
        for x1i, x2i, yi in zip(x1, x2, y):
            if x1i not in zip(x1_validation) and x2i not in zip(x2_validation) and yi not in zip(y_validation):
                w1 = w1 + lr * (yi - (x1i * w1 + x2i * w2 + b)) * x1i
                w2 = w2 + lr * (yi - (x1i * w1 + x2i * w2 + b)) * x2i
                b = b + lr * (yi - (x1i * w1 + x2i * w2 + b))

        if epoch >= 4 and epoch % 4 == 0:
            pre_validation_error = curr_validation_error
            if epoch % 100 == 0: # print only every 20 epochs
                print "epoch:\t", epoch, "\t curr_validation_error:\t", curr_validation_error
            curr_validation_error = 0

        # validation
        for x1i, x2i, yi in zip(x1_validation, x2_validation, y_validation):
            curr_validation_error += abs(yi - (x1i * w1 + x2i * w2 + b))

        epoch += 1

    print "Result: w1:\t", w1, "\tw2:\t", w2, "\tb:\t", b, "\tepoch:\t", epoch
    print "pre_validation_error:\t", pre_validation_error, "curr_validation_error:\t", curr_validation_error

In [ ]:
# online
# learning rate lr = 0.0001
lr = 0.0001
train_and_get_result(x1, x2, lr)

random initial w1:	0.590393204855
random initial w2:	2.43050498385
random initial b:	1.78049413864
epoch:	100 	 curr_validation_error:	115.432198524
epoch:	200 	 curr_validation_error:	67.5640940189
epoch:	300 	 curr_validation_error:	59.1471899489
epoch:	400 	 curr_validation_error:	56.2009213422
epoch:	500 	 curr_validation_error:	53.8838465272
epoch:	600 	 curr_validation_error:	51.5413845854
epoch:	700 	 curr_validation_error:	49.2625662521
epoch:	800 	 curr_validation_error:	47.0783342066
epoch:	900 	 curr_validation_error:	44.9935463339
epoch:	1000 	 curr_validation_error:	43.006458919
epoch:	1100 	 curr_validation_error:	41.1148864285
epoch:	1200 	 curr_validation_error:	39.3150506817
epoch:	1300 	 curr_validation_error:	37.6431544282
epoch:	1400 	 curr_validation_error:	36.063936173
epoch:	1500 	 curr_validation_error:	34.5619347194
epoch:	1600 	 curr_validation_error:	33.1392842915
epoch:	1700 	 curr_validation_error:	31.8460241792
epoch:	1800 	 curr_validation_error:	30.65047

In [ ]:
# online
# learning rate lr = 0.001
lr = 0.001
train_and_get_result(x1, x2, lr)

In [6]:
# minibatch


In [ ]:
# batch
